In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='1.12.0',image_scope='inference',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker pull 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu

In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin  763104351884.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker build -t 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-age -f Dockerfile.train .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (3/3)                                          docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 300B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
[+] Building 0.3s (8/9)                                          docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 300B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

In [3]:
!docker images

REPOSITORY                                                           TAG                        IMAGE ID       CREATED         SIZE
523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2   2.12.0-cpu-py310-2         31d25f6b915d   2 minutes ago   6.13GB
523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2   2.12.0-cpu-py310-pippo-2   31d25f6b915d   2 minutes ago   6.13GB
tensorflow-training                                                  2.12.0-cpu-py310-pippo-2   d623601d8ce4   2 hours ago     6.13GB
523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2   <none>                     d623601d8ce4   2 hours ago     6.13GB


In [16]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='tensorflow-training:2.12.0-cpu-py310-age',
    role=role,
    instance_count=1,
    instance_type='local',
    hyperparameters=hyperparameters,
    output_path='file://{}/data/output'.format(os.getcwd())
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('file://{}/data/input/train.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-10-25-15-18-23-133
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' is not installed. Proceeding to check for 'docker-compose' CLI.
INFO:sagemaker.local.image:'Docker Compose' found using Docker Compose CLI.
INFO:sagemaker.local.local_session:Starting training job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials fou

##### ESTIMATOR FIT STARTED
 algo-1-b4f8m Pulling 
 algo-1-b4f8m Error 
Error response from daemon: pull access denied for tensorflow-training, repository does not exist or may require 'docker login': denied: requested access to the resource is denied


ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-training-job


RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpiv5kcz1r/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit']. Process exited with code: 18

In [ ]:
!tar -xzf data/output/output.tar.gz

In [ ]:
!SM_MODEL_DIR="./data/output" python main.py

In [ ]:
!tar -xzf ./data/output/model.tar.gz

In [ ]:
%%sh
# Specify an image name
image_name=tensorflow-training-age
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-age"
echo "fullname: ${fullname} ######################"

aws ecr create-repository --repository-name "${image_name}"

aws ecr describe-repositories --repository-names "${image_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}" > /dev/null
fi

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2

In [ ]:
!docker tag tensorflow-training:2.12.0-cpu-py310-pippo-2 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-pippo-2

In [8]:
!docker push 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-2

The push refers to repository [523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2]

e1bcee2e: Preparing 
22878ecf: Preparing 
db4f6696: Preparing 
f529ae49: Preparing 
75f13e59: Preparing 
0c6c24f9: Preparing 
a1e64371: Preparing 
29631b13: Preparing 
3f03efd0: Preparing 
9b75d5ee: Preparing 
370de786: Preparing 
a16f1a7a: Preparing 
fc8740d1: Preparing 
292691fc: Preparing 
6672bf9d: Preparing 
607be3da: Preparing 
afc9b360: Preparing 
f934e4df: Preparing 
80a2a6eb: Preparing 
a0538fa4: Preparing 
6d109c54: Preparing 
f0a068df: Preparing 
443b7673: Preparing 
37a36861: Preparing 
ecb4bd81: Preparing 
34b60c81: Preparing 
07d5d0ab: Preparing 
e6c8e003: Preparing 
ab822c1c: Preparing 
3609315f: Preparing 
0445c449: Layer already exists 2.12.0-cpu-py310-2: digest: sha256:750a3bfe85174b7cb20266fa571ba5547ade35d7e5c77e1a721f2ad254278a7d size: 6831


In [11]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-2',
    role=role,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    hyperparameters=hyperparameters,
    output_path='s3://itsarghis/data/output'.format(os.getcwd())
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('s3://itsarghis/data/input/train.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')



INFO:sagemaker:Creating training-job with name: tensorflow-training-2-2024-10-25-14-40-00-560


##### ESTIMATOR FIT STARTED
2024-10-25 14:40:02 Starting - Starting the training job...
2024-10-25 14:40:28 Starting - Preparing the instances for training......
2024-10-25 14:41:19 Downloading - Downloading input data...
2024-10-25 14:41:39 Downloading - Downloading the training image.........
2024-10-25 14:43:35 Training - Training image download completed. Training in progress....
2024-10-25 14:44:01 Uploading - Uploading generated training model2024-10-25 14:43:52,900 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2024-10-25 14:43:52,912 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-10-25 14:43:52,937 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-10-25 14:43:52,961 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-10-25 14:43:52,985 sagemaker-training-toolkit INFO     No Neurons detected (normal i

UnexpectedStatusException: Error for Training job tensorflow-training-2-2024-10-25-14-40-00-560: Failed. Reason: AlgorithmError: NotFoundError:
ExitCode 1
ErrorMessage "FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/model/data/training/train.csv'"
Command "/usr/local/bin/python3.10 main.py --epochs 1", exit code: 1. Check troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html